## 🧠 Introduction to Neural Style Transfer

Neural Style Transfer is a computer vision technique that allows us to combine the **content of one image** with the **style of another** to produce a new, artistic image.

In this project, we use the **VGG19** convolutional neural network pre-trained on ImageNet to extract:

- **Content features** (structure and layout) from a **content image**
- **Style features** (color, texture, and brushstrokes) from a **style image**

Then, we optimize a new image to minimize a **loss function** that balances:

- **Content loss**: how different the generated image is from the content image
- **Style loss**: how different the textures and colors are from the style image

This notebook implements the entire process step-by-step, using **TensorFlow** and **Keras**.

---

### 📝 Steps Overview:

1. Load and preprocess content & style images
2. Extract feature maps from VGG19 layers
3. Compute Gram matrices for style
4. Define loss functions
5. Optimize an image starting from the content image
6. Visualize stylized results



-----

## 📦 Step 1: Import Dependencies

We begin by importing essential Python libraries:

- **TensorFlow**: For deep learning and model operations (VGG19)
- **NumPy**: For numerical operations on image tensors
- **Matplotlib**: To visualize images during training
- **PIL (Python Imaging Library)**: To load and manipulate images
- **Warnings/OS**: To suppress unnecessary logs and keep the notebook clean


In [ ]:
# 📦 Import Required Libraries

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import warnings

# Suppress TensorFlow and Python warnings for cleaner output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TF messages: 0=all, 1=ignore INFO, 2=ignore WARNING, 3=ignore ERROR
warnings.filterwarnings('ignore')


-----

## 🖼️ Step 2: Load and Preprocess Images

We define a utility function `load_image()` that:

- Opens the image using **PIL**
- Converts it to **RGB**
- Resizes it to a maximum dimension (default: 300px) while maintaining the aspect ratio
- Normalizes the pixel values to the [0, 1] range
- Adds a batch dimension so it can be fed into the VGG19 model

This ensures both the content and style images are in the correct format for processing.


In [ ]:
def load_image(path_to_img, max_dim=300):
    img = Image.open(path_to_img)                  # Open the image file
    img = img.convert('RGB')                       # Ensure it's in RGB format
    img.thumbnail((max_dim, max_dim))              # Resize while preserving aspect ratio
    img = np.array(img)                            # Convert to NumPy array
    img = img[tf.newaxis, ...] / 255.0             # Add batch dimension & normalize [0,1]
    return tf.convert_to_tensor(img, dtype=tf.float32)  # Convert to TensorFlow tensor

In [ ]:
content_image = load_image("content.jpg")
style_image = load_image("style.jpg")

In [ ]:
# Load pretrained VGG19 without the fully connected layers (for feature extraction)
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet') 

vgg.trainable = False  # Freeze the model weights; do not update during training